In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Images.shape()

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image as pil
#import tensorflow.keras.layers as Layers
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import utils as np_utils
from keras.utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

train_df = pd.read_csv('../input/shopee-product-detection-open/train.csv')
test_df = pd.read_csv('../input/shopee-product-detection-open/test.csv')

# Reading Data

In [ ]:
dir = '../input/shopee-product-detection-open/train/train/train/'
Images = []
Labels = []
#n = 1500 # No. of images to read for each category  #  Total no. of images = (n*42)

for labels in os.listdir(dir):
    #i = 0
    for file in os.listdir(dir + labels):
        #if i > n-1:
        #    break
        path = dir + labels + '/' + file
        
        # Reading image as grayscale and resizing
        image = cv2.imread(path,0)
        image = cv2.resize(image,(100,100))
        image = np.expand_dims(image, 2)
        #i += 1
        
        Images.append(image)
        Labels.append(labels)
    

# Using ImageDataGenerator

In [ ]:
train_data_dir = '../input/shopee-product-detection-open/train/train/train'
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=45,
                                   width_shift_range=.15,
                                   height_shift_range=.15,
                                   horizontal_flip=True,
                                   zoom_range=0.5,
                                   validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(50, 50),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(train_data_dir, # same directory as training data
                                                         target_size=(50, 50),
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         subset='validation') # set as validation data

# Split Data

In [ ]:
#train_list = [Images,Labels]

# Converting Images and Labels to array
Images_array = np.array(Images)
Labels_array = np.array(Labels)

# Rescaling
Images_array = Images_array/255

# Split Train and Test Data
X_train, X_test, y_train, y_test = train_test_split(Images_array, 
                                                    Labels_array, 
                                                    random_state=42, 
                                                    test_size=0.3, 
                                                    stratify=Labels_array, 
                                                    shuffle=True)

# One-hot encoding column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Compiling Model

In [ ]:
model = Sequential()

# Adding Layers to Model
model.add(Conv2D(128, kernel_size=3, activation="relu", input_shape=(100,100,1)))
model.add(Conv2D(128, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
Dropout(.2)
model.add(Conv2D(256, kernel_size=3, activation="relu"))
model.add(Conv2D(256, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
Dropout(.2)
model.add(Conv2D(512, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
Dropout(.2)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(42, activation='softmax'))

# Compiling Model
model.compile(loss='categorical_crossentropy',
              optimizer='Adam', 
              metrics=['accuracy'])

#model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Adding weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Adding weights to categories with lesser data
y_integers = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))

# Model fit

In [ ]:
model.fit(train_generator, 
          epochs=6, 
          validation_data=validation_generator, 
          class_weight=d_class_weights)

# Reading Test images

In [ ]:
dir = '../input/shopee-product-detection-open/test/test/test/'
Images_name = []
Images_test_array = []

#n = 100 # No. of test images to read for test

i = 0
for file in test_df['filename']:
    #if i > n-1 :
    #    break
    
    path = dir + '/' +file
    image = cv2.imread(path,0)
    image = cv2.resize(image,(50,50))
    image = np.expand_dims(image, 2)
    
    i += 1
    Images_name.append(file)
    Images_test_array.append(image)

# Rescaling 
Images_test_array = np.array(Images_test_array)/255

# Model predict

In [ ]:
pred_category = model.predict_classes(Images_test_array)

# Output Dataframe answer as CSV

In [ ]:
ans_df = pd.DataFrame({'filename':Images_name, 'category': pred_category})
ans_df['category'] = ans_df['category'].astype(str)
ans_df.to_csv("/kaggle/working/submission.csv", index=False)